In [1]:
from torch import nn
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

import torch
torch.cuda.empty_cache()

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold

In [5]:
models_dir = "models/bert-k-fold"

In [6]:
labels = open('data/classes.txt').read().splitlines()
df = pd.read_csv("data/belief_benchmark_all_train.csv")
df.head()

,text,sentiment,label
0,Private-sector actors expect their business pa...,NEGATIVE,1
1,Lack of product uniformity is perceived by con...,"NEGATIVE, NEGATIVE, NEGATIVE, NEGATIVE",1
2,"However , to the extent that the empirical str...",POSITIVE,2
3,Firms prefer to contract with producers with w...,POSITIVE,2
4,Farmers attempted to use alternate wetting and...,POSITIVE,2


In [7]:
transformer_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(transformer_name)
# NOTE: for cross validation, the model should be initialized inside the cv loop

In [8]:
def tokenize(batch):
    return tokenizer(batch['text'], truncation=True)

In [9]:
def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    report = metrics.classification_report(y_true, y_pred)
    print("report: \n", report)
    
    print("rep type: ", type(report))
    

    return {'f1':metrics.f1_score(y_true, y_pred, average="macro")}

In [10]:
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        cls_outputs = outputs.last_hidden_state[:, 0, :]
        cls_outputs = self.dropout(cls_outputs)
        logits = self.classifier(cls_outputs)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [11]:
# this is for creating cross-validation folds
def get_sample_based_on_idx(data, indeces):
    return data.iloc[indeces, :].reset_index()

In [12]:
# defining hyperparams
num_epochs = 8
batch_size = 6
weight_decay = 0.01
print(f"num_epochs: {num_epochs}, batch_size: {batch_size}, weight_decay: {weight_decay}")
training_args = TrainingArguments(
    output_dir="./results_sentiment_analysis",
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    weight_decay=weight_decay,
    load_best_model_at_end=True, # this is supposed to make sure the best model is loaded by the trainer at the end
    metric_for_best_model="eval_f1" 
    )

num_epochs: 8, batch_size: 6, weight_decay: 0.01


In [13]:
# specify what percent of the all_train data should be used in cross-fold validation 
# we can use this to create a learning curve, e.g., performance with 50, 80, 90% of data
train_size = 0.8

if train_size < 1.0:
    df, _ = train_test_split(df, test_size=1.0-train_size, random_state=1, stratify=df[['label']])
print(f"train_size: {train_size} (total samples: {len(df)})")

train_size: 0.8 (total samples: 173)


In [14]:
output = open("original.txt", "a")

fold = 0
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
for train_df_idx, eval_df_idx in kfold.split(df):
    
    print(f"************** BEGIN FOLD: {fold+1} **************")
    output.write(f"FOLD: {fold}\n")
    new_df = pd.DataFrame()
    
    train_df = get_sample_based_on_idx(df, train_df_idx)
    print("LEN DF: ", len(train_df))
    output.write(f"LEN DF: {len(train_df)}\n")

    print("done train df")
    output.write("done train df\n")
    eval_df = get_sample_based_on_idx(df, eval_df_idx)

    print("done eval df")
    output.write("done eval df\n")
    print("LEN EVAL: ", len(eval_df))
    output.write(f"LEN EVAL: {len(eval_df)}\n")

    ds = DatasetDict()
    ds['train'] = Dataset.from_pandas(train_df)
    ds['validation'] = Dataset.from_pandas(eval_df)
    train_ds = ds['train'].map(
        tokenize, batched=True
    )
    eval_ds = ds['validation'].map(
        tokenize,
        batched=True
    )

    model = AutoModelForSequenceClassification.from_pretrained(transformer_name, num_labels=4)
    tokenizer = AutoTokenizer.from_pretrained(transformer_name)

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        tokenizer=tokenizer,
    )
    trainer.train()
    # after training, predict (will use best model?)
    preds = trainer.predict(eval_ds)
#     print("HERE: " , preds)
    final_preds = [np.argmax(x) for x in preds.predictions]
    real_f1 = metrics.f1_score(final_preds, eval_df["label"], average="macro")
    print("F-1: ", real_f1)
    output.write(f"F-1: {real_f1}\n")
    model_name = f"{transformer_name}-best-of-fold-{fold}-f1-{real_f1}"
    model_dir = os.path.join(models_dir, model_name)

    trainer.save_model(model_dir)

    for i, item in enumerate(final_preds):
        if item != eval_ds["label"][i]:
            wrong_df = pd.DataFrame()
            wrong_df["text"] = [eval_df["text"][i]]
            wrong_df["real"] = [eval_df["label"][i]]
            wrong_df["predicted"] = [item]
            new_df = pd.concat([new_df, wrong_df])

    new_df.to_csv(f"{models_dir}/wrong_predictions_{fold}.csv")

    print(f"************** END FOLD: {fold+1} **************\n")
    fold += 1
        

************** BEGIN FOLD: 1 **************
LEN DF:  138
done train df
done eval df
LEN EVAL:  35


Map:   0%|          | 0/138 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/184 [00:00<?, ?it/s]

...

c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


report: 
               precision    recall  f1-score   support

           0       0.69      0.60      0.64        15
           1       0.57      0.67      0.62         6
           2       0.50      0.78      0.61         9
           3       0.00      0.00      0.00         4

    accuracy                           0.59        34
   macro avg       0.44      0.51      0.47        34
weighted avg       0.54      0.59      0.55        34

rep type:  <class 'str'>
{'eval_loss': 1.5565199851989746, 'eval_f1': 0.4667343526039178, 'eval_runtime': 0.241, 'eval_samples_per_second': 141.078, 'eval_steps_per_second': 24.896, 'epoch': 8.0}
{'train_runtime': 67.4981, 'train_samples_per_second': 16.475, 'train_steps_per_second': 2.845, 'train_loss': 0.4454421599706014, 'epoch': 8.0}


  0%|          | 0/6 [00:00<?, ?it/s]

c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


report: 
               precision    recall  f1-score   support

           0       0.63      0.80      0.71        15
           1       0.75      0.50      0.60         6
           2       0.55      0.67      0.60         9
           3       0.00      0.00      0.00         4

    accuracy                           0.62        34
   macro avg       0.48      0.49      0.48        34
weighted avg       0.56      0.62      0.58        34

rep type:  <class 'str'>
F-1:  0.4764705882352941
************** END FOLD: 5 **************



In [15]:
print("\n******************* holdout results ******************* ")
holdout_df = df = pd.read_csv("data/belief_benchmark_holdout.csv")
holdout_ds = Dataset.from_pandas(holdout_df)
holdout_ds = holdout_ds.map(tokenize, batched=True)

preds = trainer.predict(holdout_ds)


******************* holdout results ******************* 


Map:   0%|          | 0/146 [00:00<?, ? examples/s]

  0%|          | 0/25 [00:00<?, ?it/s]

report: 
               precision    recall  f1-score   support

           0       0.53      0.80      0.63        49
           1       0.67      0.38      0.49        26
           2       0.65      0.63      0.64        59
           3       0.00      0.00      0.00        12

    accuracy                           0.59       146
   macro avg       0.46      0.45      0.44       146
weighted avg       0.56      0.59      0.56       146

rep type:  <class 'str'>


c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
final_preds = [np.argmax(x) for x in preds.predictions]
real_f1 = metrics.f1_score(final_preds, holdout_df["label"], average="macro")
print("F-1: ", real_f1)
y_pred = []
for i, item in enumerate(final_preds):
    y_pred.append(item)

y_true = holdout_ds["label"]


F-1:  0.4399705634987384


In [17]:
print(f"y_true: {y_true}")
print(f"y_pred: {y_pred}")
import pickle
pickle.dump(y_pred, open(f"few_shot_results/y_pred_{int(train_size*100)}", "wb"))
pickle.dump(y_true, open(f"few_shot_results/y_true_{int(train_size*100)}", "wb"))

y_true: [0, 2, 2, 0, 0, 1, 1, 2, 2, 2, 2, 1, 0, 1, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 1, 2, 1, 0, 2, 0, 0, 3, 3, 0, 2, 2, 1, 1, 0, 1, 1, 2, 0, 2, 2, 2, 1, 2, 0, 3, 2, 2, 1, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 2, 0, 3, 2, 2, 3, 0, 0, 2, 0, 2, 2, 2, 2, 1, 1, 0, 0, 2, 3, 1, 0, 2, 0, 0, 2, 1, 3, 2, 2, 1, 2, 3, 2, 0, 2, 1, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 2, 2, 0, 2, 3, 2, 0, 2, 2, 2, 0, 3, 3, 2, 0, 2, 1, 2, 0, 0, 0, 2]
y_pred: [0, 0, 2, 0, 0, 1, 0, 2, 0, 2, 2, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 1, 0, 1, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 1, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 1, 1, 2, 2, 2, 0, 1, 2, 2, 2, 2, 1, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 1, 2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 1, 2, 2, 0, 0, 2]


In [18]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
output.write("{torch.cuda.memory_summary(device=None, abbreviated=False)}")

output.close()